In [1]:
# ✅ Real-Time Sentiment Analysis for Customer Feedback (BERT + CPU Version)
# ==========================================================================
# Dataset: mobile_reviews_sentiment.csv (from Kaggle)
# Columns: review_text, sentiment

# -------------------------------
# 1️⃣ Install (run once if needed)
# -------------------------------
# !pip install torch torchvision torchaudio transformers scikit-learn pandas numpy tqdm nltk nlpaug

# -------------------------------
# 2️⃣ Import Libraries
# -------------------------------
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

# -------------------------------
# 3️⃣ Load & Clean Dataset
# -------------------------------
df = pd.read_csv("C:/Users/HP/Desktop/kani/Data_science/guvi/Project/NLP/mobile_reviews_sentiment.csv")

# Keep relevant columns
df = df[['review_text', 'sentiment']]

# Drop missing values
df.dropna(subset=['review_text', 'sentiment'], inplace=True)

# Convert sentiment to lowercase for consistency
df['sentiment'] = df['sentiment'].str.lower()

# Clean text
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['clean_text'] = df['review_text'].apply(clean_text)

# Encode sentiment labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])
print("Encoded classes:", le.classes_)
print("\nSentiment distribution:\n", df['sentiment'].value_counts())

# -------------------------------
# 4️⃣ Train-Test Split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Convert to clean Series
X_train = pd.Series(X_train).fillna("").astype(str)
X_test  = pd.Series(X_test).fillna("").astype(str)
y_train = y_train.to_numpy()
y_test  = y_test.to_numpy()

# -------------------------------
# 5️⃣ Tokenization
# -------------------------------
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_texts(texts, tokenizer, max_len=128):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )

train_encodings = encode_texts(X_train, tokenizer)
test_encodings  = encode_texts(X_test, tokenizer)

# -------------------------------
# 6️⃣ PyTorch Dataset
# -------------------------------
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, y_train)
test_dataset  = ReviewDataset(test_encodings, y_test)

# -------------------------------
# 7️⃣ Dataloaders
# -------------------------------
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

# -------------------------------
# 8️⃣ Model Setup (CPU Version)
# -------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on:", device)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# -------------------------------
# 9️⃣ Training Loop
# -------------------------------
model.train()
for epoch in range(2):  # start with 2 epochs for CPU, can increase later
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Average Loss: {total_loss/len(train_loader):.4f}")

# -------------------------------
# 🔟 Evaluation
# -------------------------------
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds.extend(torch.argmax(outputs.logits, axis=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\n--- Evaluation Report ---")
print(classification_report(true_labels, preds, target_names=le.classes_))

# -------------------------------
# 11️⃣ Save Model
# -------------------------------
torch.save(model.state_dict(), "bert_sentiment_mobile_reviews.pt")
print("✅ Model saved successfully!")

# -------------------------------
# ✅ 12️⃣ Improved Test Predictions (with Correct Label Mapping)
# -------------------------------

# ✅ Check and display label mapping
print("\n--- Label Mapping ---")
for i, label in enumerate(le.classes_):
    print(f"LABEL_{i}: {label}")

sample_texts = [
    "The product quality is outstanding and delivery was fast!",
    "It’s okay, not too great but not bad either.",
    "Worst experience ever, totally waste of money."
]

# Encode test sentences
encoded = encode_texts(sample_texts, tokenizer)
encoded = {k: v.to(device) for k, v in encoded.items()}

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**encoded)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, axis=1).cpu().numpy()

# Decode predictions using label encoder
decoded_preds = le.inverse_transform(preds)

print("\n--- Predictions ---")
for text, label, prob in zip(sample_texts, decoded_preds, probs):
    confidence = prob.max().item()
    print(f"\n{text}\n→ Predicted Sentiment: {label.upper()} (Confidence: {confidence:.4f})")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Encoded classes: ['negative' 'neutral' 'positive']

Sentiment distribution:
 sentiment
positive    27540
neutral     12549
negative     9911
Name: count, dtype: int64
Training on: cpu


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:   4%|██▊                                                                 | 210/5000 [03:47<1:26:17,  1.08s/it]


KeyboardInterrupt: 

In [2]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
res = pipe("The product quality is outstanding and delivery was fast!")[0]

for r in res:
    print(f"{r['label']} -> {r['score']:.4f}")


Device set to use cpu


LABEL_0 -> 0.9999
LABEL_1 -> 0.0000
LABEL_2 -> 0.0000


C:\Users\HP\anaconda3\envs\bert_env\lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
for text, pred in zip(sample_texts, preds):
    label = le.inverse_transform([pred])[0]
    print(f"\n{text} --> Predicted Sentiment: {label}")



The product quality is outstanding and delivery was fast! --> Predicted Sentiment: negative

It’s okay, not too great but not bad either. --> Predicted Sentiment: neutral

Worst experience ever, totally waste of money. --> Predicted Sentiment: negative


In [7]:
# ✅ FAST VERSION — Real-Time Sentiment Analysis (DistilBERT + CPU)
# ==================================================================
# Uses only 1,000 samples for quick testing
# Great for debugging, learning, and label verification

# -------------------------------
# 1️⃣ Install (run once if needed)
# -------------------------------
# !pip install torch torchvision torchaudio transformers scikit-learn pandas numpy tqdm nltk

# -------------------------------
# 2️⃣ Import Libraries
# -------------------------------
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

# -------------------------------
# 3️⃣ Load & Clean Dataset (Reduced Size)
# -------------------------------
df = pd.read_csv("C:/Users/HP/Desktop/kani/Data_science/guvi/Project/NLP/mobile_reviews_sentiment.csv")
df = df[['review_text', 'sentiment']].dropna()

# sample only 1k for faster training
df = df.sample(1000, random_state=42)

# clean text
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['clean_text'] = df['review_text'].apply(clean_text)

# encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'].str.lower())
print("Encoded classes:", le.classes_)

# -------------------------------
# 4️⃣ Train-Test Split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

# -------------------------------
# 5️⃣ Tokenization (Shorter max_len)
# -------------------------------
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode_texts(texts, tokenizer, max_len=64):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=max_len, return_tensors='pt')

train_encodings = encode_texts(X_train, tokenizer)
test_encodings  = encode_texts(X_test, tokenizer)

# -------------------------------
# 6️⃣ Dataset + Dataloader
# -------------------------------
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)

train_loader = DataLoader(ReviewDataset(train_encodings, y_train.to_numpy()), batch_size=8, shuffle=True)
test_loader  = DataLoader(ReviewDataset(test_encodings,  y_test.to_numpy()),  batch_size=8, shuffle=False)

# -------------------------------
# 7️⃣ Model Setup
# -------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on:", device)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3).to(device)
optimizer = AdamW(model.parameters(), lr=3e-5)

# ✅ Fix label mapping for sentiment classes
model.config.id2label = {
    0: "negative",
    1: "neutral",
    2: "positive"
}
model.config.label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}


# -------------------------------
# 8️⃣ Quick Training (1 Epoch)
# -------------------------------
model.train()
for epoch in range(4):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Avg Loss: {total_loss/len(train_loader):.4f}")

# -------------------------------
# 9️⃣ Evaluation
# -------------------------------
model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds.extend(torch.argmax(outputs.logits, axis=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\n--- Evaluation Report ---")
print(classification_report(true_labels, preds, target_names=le.classes_))

# -------------------------------
# 🔟 Predictions with Correct Labels
# -------------------------------
print("\n--- Label Mapping ---")
for i, label in enumerate(le.classes_):
    print(f"LABEL_{i}: {label}")

sample_texts = [
    "The product quality is outstanding and delivery was fast!",
    "It’s okay, not too great but not bad either.",
    "Worst experience ever, totally waste of money."
]

encoded = encode_texts(sample_texts, tokenizer)
encoded = {k: v.to(device) for k, v in encoded.items()}

with torch.no_grad():
    outputs = model(**encoded)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, axis=1).cpu().numpy()

print("\n--- Predictions ---")
for text, pred_idx, prob in zip(sample_texts, preds, probs):
    confidence = prob.max().item()
    sentiment = model.config.id2label[pred_idx]
    print(f"\n{text}\n→ Predicted Sentiment: {sentiment.upper()} (Confidence: {confidence:.4f})")

model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Encoded classes: ['negative' 'neutral' 'positive']
Training on: cpu


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.13it/s]


Epoch 1 Avg Loss: 0.2537


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.12it/s]


...

Epoch 4: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.08it/s]


Epoch 4 Avg Loss: 0.0018


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.60it/s]



--- Evaluation Report ---
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00        41
     neutral       1.00      1.00      1.00        51
    positive       1.00      1.00      1.00       108

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200


--- Label Mapping ---
LABEL_0: negative
LABEL_1: neutral
LABEL_2: positive

--- Predictions ---

The product quality is outstanding and delivery was fast!
→ Predicted Sentiment: POSITIVE (Confidence: 0.8753)

It’s okay, not too great but not bad either.
→ Predicted Sentiment: NEUTRAL (Confidence: 0.9761)

Worst experience ever, totally waste of money.
→ Predicted Sentiment: NEGATIVE (Confidence: 0.9460)


('./sentiment_model\\tokenizer_config.json',
 './sentiment_model\\special_tokens_map.json',
 './sentiment_model\\vocab.txt',
 './sentiment_model\\added_tokens.json')

In [2]:
model.config.id2label
model.config.label2id


{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}

In [8]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

model_path = "./sentiment_model"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()

text = "Worst experience ever"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=1)
print(probs)


tensor([[0.8965, 0.0318, 0.0717]], grad_fn=<SoftmaxBackward0>)
